## 심층 신경망 구현하기

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
# [털, 날개]
# Tarket : 어떤 종류의 동물인지.
x_data = np.array( [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# Tarket : 원핫인코딩(one-hot encoding)
# 데이터가 가질 수 있는 값들을 일렬로 나열한 배열로 만들고, 
# 그중의 표현하려는 값을 인덱스의 원소만 1로 표현하고, 나머지는 0으로 채우는 표기법

# [털, 날개]


# [기타, 포유류, 조류]
# 다음과 같은 형식을 one-hot 형식의 데이터라고 합니다.
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [0]:
X = tf.placeholder(tf.float32)   #  X에 들어갈 값(공간)
Y = tf.placeholder(tf.float32)   #  Y에 들어갈 값(공간)

In [0]:
# W1 : [2,10] -> [특징, 은닉층의 뉴런 수]
# W2 : [10,3] -> [은닉층의 뉴런 수, 분류 수]
W1 = tf.Variable(tf.random_uniform([2,10],-1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([10,3],-1.0, 1.0))

# 편향
# b1 = [10] -> 은닉층의 뉴런 수
# b2 = [3]  -> 분류 수 
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

In [0]:
# 첫 번째 가중치와 편향, 활성화 함수를 적용 
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [0]:
# 두 번째 가중치와 편향,
model = tf.add(tf.matmul(L1, W2), b2)

In [0]:
# 교차 엔트로피 함수 사용
# 최적화 함수 : AdamOptimizer 함수
# AdamOptimizer는 GradientDescentOptimizer보다 보편적으로 성능이 더 좋다.
# Tf가 제공하는 다양한 최적화 함수 : https://www.tensorflow.org/api_guides/python/train
cost = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model)) # 버전 변경
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [29]:
# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 레이블 데이터를 이용하여 학습을 진행
for step in range(100):
  sess.run(train_op,feed_dict={X:x_data, Y:y_data})
  
  # 학습도중 10번씩 손실값을 출력
  if (step+1)%10 == 0:
    print(step+1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))


10 0.9271534
20 0.6812835
30 0.51234275
40 0.3851327
50 0.28330165
60 0.19858724
70 0.13136534
80 0.08565382
90 0.057365503
100 0.040310636


In [32]:
#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
